In [1]:
from pyspark import SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, map_keys, col, first, get_json_object

In [2]:
# Spark 세션 생성
spark = SparkSession \
        .builder \
        .appName("processingJson") \
        .getOrCreate()

23/11/17 13:32:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
# 분할된 JSON 파일 경로 선택하는 함수
def nth_json_path(n):
    return f'/Users/b06/Desktop/yeardream/medi-05/data/naverplace_meta/naverplace_meta_{n}.json'

In [4]:
# 첫번째 JSON 파일 경로
json_path_1 = nth_json_path(1)
json_path_1

'/Users/b06/Desktop/yeardream/medi-05/data/naverplace_meta/naverplace_meta_1.json'

In [5]:
data = spark.read.json(json_path_1)

23/11/17 13:32:25 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
## EDA
#df.show(1)
#df.printSchema(1)
#df.dtypes
columns = data.columns

In [7]:
hb_cols = [c for c in columns if "HospitalBase" in c]
hb_cols

['HospitalBase:11779766',
 'HospitalBase:1233961055',
 'HospitalBase:12857046',
 'HospitalBase:1344046290',
 'HospitalBase:1413208872',
 'HospitalBase:1934828030',
 'HospitalBase:19518318',
 'HospitalBase:19530874',
 'HospitalBase:265615453',
 'HospitalBase:33237939']

In [8]:
df = data.select(*hb_cols)
df.printSchema(1)
# df.printSchema(2)

root
 |-- HospitalBase:11779766: struct (nullable = true)
 |-- HospitalBase:1233961055: struct (nullable = true)
 |-- HospitalBase:12857046: struct (nullable = true)
 |-- HospitalBase:1344046290: struct (nullable = true)
 |-- HospitalBase:1413208872: struct (nullable = true)
 |-- HospitalBase:1934828030: struct (nullable = true)
 |-- HospitalBase:19518318: struct (nullable = true)
 |-- HospitalBase:19530874: struct (nullable = true)
 |-- HospitalBase:265615453: struct (nullable = true)
 |-- HospitalBase:33237939: struct (nullable = true)



In [9]:
targets = [
    'id', 
    'name', 
    'road', 
    'reviewSettings', 
    'conveniences', 
    'keywords', 
    'phone', 
    'virtualPhone', 
    'naverBookingUrl', 
    'talktalkUrl', 
    'paymentInfo', 
    'homepages',
    'visitorReviewsTotal',
    'description',
    'Images'
]

In [25]:
sel_cols = [col(hb + "." + t).alias(hb + "_" + t) for hb in hb_cols for t in targets]
sel_cols

[Column<'HospitalBase:11779766.id AS `HospitalBase:11779766_id`'>,
 Column<'HospitalBase:11779766.name AS `HospitalBase:11779766_name`'>,
 Column<'HospitalBase:11779766.road AS `HospitalBase:11779766_road`'>,
 Column<'HospitalBase:11779766.reviewSettings AS `HospitalBase:11779766_reviewSettings`'>,
 Column<'HospitalBase:11779766.conveniences AS `HospitalBase:11779766_conveniences`'>,
 Column<'HospitalBase:11779766.keywords AS `HospitalBase:11779766_keywords`'>,
 Column<'HospitalBase:11779766.phone AS `HospitalBase:11779766_phone`'>,
 Column<'HospitalBase:11779766.virtualPhone AS `HospitalBase:11779766_virtualPhone`'>,
 Column<'HospitalBase:11779766.naverBookingUrl AS `HospitalBase:11779766_naverBookingUrl`'>,
 Column<'HospitalBase:11779766.talktalkUrl AS `HospitalBase:11779766_talktalkUrl`'>,
 Column<'HospitalBase:11779766.paymentInfo AS `HospitalBase:11779766_paymentInfo`'>,
 Column<'HospitalBase:11779766.homepages AS `HospitalBase:11779766_homepages`'>,
 Column<'HospitalBase:11779766

In [26]:
sel_df = df.select(*sel_cols)
sel_df.printSchema(2)

root
 |-- HospitalBase:11779766_id: string (nullable = true)
 |-- HospitalBase:11779766_name: string (nullable = true)
 |-- HospitalBase:11779766_road: string (nullable = true)
 |-- HospitalBase:11779766_reviewSettings: struct (nullable = true)
 |    |-- __typename: string (nullable = true)
 |    |-- blog: long (nullable = true)
 |    |-- cafe: long (nullable = true)
 |    |-- keyword: string (nullable = true)
 |    |-- showVisitorReviewScore: boolean (nullable = true)
 |-- HospitalBase:11779766_conveniences: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- HospitalBase:11779766_keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- HospitalBase:11779766_phone: string (nullable = true)
 |-- HospitalBase:11779766_virtualPhone: string (nullable = true)
 |-- HospitalBase:11779766_naverBookingUrl: string (nullable = true)
 |-- HospitalBase:11779766_talktalkUrl: string (nullable = true)
 |-- HospitalBase:11779766_paymentInfo: array

In [34]:
sel_df.select('HospitalBase:11779766_id').show()

+------------------------+
|HospitalBase:11779766_id|
+------------------------+
|                11779766|
|                    NULL|
|                    NULL|
|                    NULL|
|                    NULL|
|                    NULL|
|                    NULL|
|                    NULL|
|                    NULL|
|                    NULL|
+------------------------+

